# Prepare (import library and prepare data)

In [ ]:
! pip install transformers

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import torchaudio
from sklearn.model_selection import train_test_split
import os
import sys

pd.options.mode.chained_assignment = None # avoids assignment warning
import random
from glob import glob
tqdm.pandas()  # enable progress bars in pandas operations
import gc
import cv2
import librosa
import sklearn
import json
import argparse

# Import for visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import librosa.display as lid
import IPython.display as ipd

# from kaggle_datasets import KaggleDatasets
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import transformers

from transformers import set_seed
from transformers import ASTFeatureExtractor
from transformers import ASTPreTrainedModel, ASTModel, AutoConfig, ASTConfig
import seaborn as sns


from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

import os
import sys
import csv

ModuleNotFoundError: ignored

## config

In [ ]:
import os

class CFG:
    # Debugging
    debug = False
    
    # Plot training history
    training_plot = True
    
    # Weights and Biases logging
    wandb = True
    competition   = 'birdclef-2023' 
    _wandb_kernel = 'awsaf49'
    
    # Experiment name and comment
    exp_name = 'baseline-v2'
    comment = 'EfficientNetB0|FSR|t=10s|128x384|up_thr=50|cv_filter'
    
    # Notebook link
    notebook_link = 'https://www.kaggle.com/awsaf49/birdclef23-effnet-fsr-cutmixup-train/edit'
    
    # Verbosity level
    verbose = 0
    
    # Device and random seed
    device = 'TPU-VM'
    seed = 42
    
    # Input image size and batch size
    img_size = [128, 384]
    batch_size = 32
    upsample_thr = 50 # min sample of each class (upsample)
    cv_filter = True # always keeps low sample data in train
    
    # Inference batch size, test time augmentation, and drop remainder
    infer_bs = 2
    tta = 1
    drop_remainder = True
    
    # Number of epochs, model name, and number of folds
    epochs = 25
    model_name = 'EfficientNetB0'
    fsr = True # reduce stride of stem block
    num_fold = 5
    
    # Selected folds for training and evaluation
    selected_folds = [0]

    # Pretraining, neck features, and final activation function
    pretrain = 'imagenet'
    neck_features = 0
    final_act = 'softmax'
    
    # Learning rate, optimizer, and scheduler
    lr = 1e-3
    scheduler = 'cos'
    optimizer = 'Adam' # AdamW, Adam
    
    # Loss function and label smoothing
    loss = 'BCE' # BCE, CCE
    label_smoothing = 0.05 # label smoothing
    
    # Audio duration, sample rate, and length
    duration = 10 # second
    sample_rate = 32000
    target_rate = 8000
    audio_len = duration*sample_rate
    
    # STFT parameters
    nfft = 2048
    window = 2048
    hop_length = audio_len // (img_size[1] - 1)
    fmin = 20
    fmax = 16000
    normalize = True
    
    # Data augmentation parameters
    augment=True
    
    # Spec augment
    spec_augment_prob = 0.80
    
    mixup_prob = 0.65
    mixup_alpha = 0.5
    
    cutmix_prob = 0.0
    cutmix_alpha = 0.5
    
    mask_prob = 0.65
    freq_mask = 20
    time_mask = 30


    # Audio Augmentation Settings
    audio_augment_prob = 0.5
    
    timeshift_prob = 0.0
    
    gn_prob = 0.35

    # Data Preprocessing Settings
    base_path = '/kaggle/input/birdclef-2023'  # for server: base_path = '/data/zjh_data/program/ml_project_birdclef23/birdclef-2023'
    if not os.path.exists(base_path):
        base_path = '/data/zjh_data/program/ml_project_birdclef23/birdclef-2023'
    class_names = sorted(os.listdir('{}/train_audio'.format(base_path)))
    num_classes = len(class_names)
    class_labels = list(range(num_classes))
    label2name = dict(zip(class_labels, class_names))
    name2label = {v:k for k,v in label2name.items()}

    # Training Settings
    target_col = ['target']
    tab_cols = ['filename']
    monitor = 'auc'
    
    ### add by plathzheng
    unilm_model_path = './pretrained_models/unilm/BEATs_iter3_plus_AS2M.pt'
    use_apex = True
    time_length = 10 # beats模型中，训练时，截取的音频片段时长
    ast_fix_layer = 3 # the parameters in layer<ast_fix_layer would be fixed, choosen from [0, 5], if ast_fix_layer>5 all param woudl be fixed

In [ ]:
from transformers import set_seed
set_seed(CFG.seed)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
if CFG.debug:
    device = torch.device('cpu')
    CFG.use_apex = False

# Data Analyze and Visualization

In [ ]:
BASE_PATH = '/kaggle/input/birdclef-2023'
GCS_PATH = BASE_PATH

test_paths = glob('/kaggle/input/birdclef-2023/test_soundscapes/*ogg')
test_df = pd.DataFrame(test_paths, columns=['filepath'])
test_df['filename'] = test_df.filepath.map(lambda x: x.split('/')[-1].replace('.ogg',''))

df = pd.read_csv(f'{CFG.base_path}/train_metadata.csv')
df['filepath'] = GCS_PATH + '/train_audio/' + df.filename
df['target'] = df.primary_label.map(CFG.name2label)
df.head(2)

## Utility

In [ ]:
def load_audio(filepath):
    audio, sr = librosa.load(filepath)
    return audio, sr

def get_spectrogram(audio):
    spec = librosa.feature.melspectrogram(y=audio, sr=CFG.sample_rate, 
                                   n_mels=CFG.img_size[0],
                                   n_fft=CFG.nfft,
                                   hop_length=CFG.hop_length,
                                   fmax=CFG.fmax,
                                   fmin=CFG.fmin,
                                   )
    spec = librosa.power_to_db(spec, ref=np.max)
    return spec

def display_audio(row):
    # Caption for viz
    caption = f'Id: {row.filename} | Name: {row.common_name} | Sci.Name: {row.scientific_name} | Rating: {row.rating}'
    # Read audio file
    audio, sr = load_audio(row.filepath)
    # Keep fixed length audio
    audio = audio[:CFG.audio_len]
    # Spectrogram from audio
    spec = get_spectrogram(audio)
    # Display audio
    print("# Audio:")
    display(ipd.Audio(audio, rate=CFG.sample_rate))
    # print("# Image:")
    # show_image(row.common_name)
    print('# Visualization:')
    fig, ax = plt.subplots(2, 1, figsize=(12, 2*3), sharex=True, tight_layout=True)
    fig.suptitle(caption)
    # Waveplot
    lid.waveshow(audio,
                 sr=CFG.sample_rate,
                 ax=ax[0])
    # Specplot
    lid.specshow(spec, 
                 sr = CFG.sample_rate, 
                 hop_length = CFG.hop_length,
                 n_fft=CFG.nfft,
                 fmin=CFG.fmin,
                 fmax=CFG.fmax,
                 x_axis = 'time', 
                 y_axis = 'mel',
                 cmap = 'coolwarm',
                 ax=ax[1])
    ax[0].set_xlabel('');
    fig.show()

## Check

In [ ]:
stat = df.primary_label.value_counts().index.tolist()
class_names = stat[:3] + stat[-3:] # popular + not popular
print(class_names)

class_name = class_names[0]
print(f'# Category: {class_name}')
class_df = df.query("primary_label==@class_name")
print(f'# Num Samples: {len(class_df)}')
row = class_df.sample(1).squeeze()

# Display audio
display_audio(row)

## The overview of all data

data class statistics, data class unbalanced figure, 

In [2]:
data = pd.read_csv("/kaggle/input/birdclef-2023/train_metadata.csv",engine='python')
data.head()

FileNotFoundError: ignored

## Visualization
1. show a bird image in dataset (plan to do by zjh)
1. feature visualization: data audio wav show, data spectrogram show, data mfcc show, data cross zero rate feature show (zjh)
2. statistic (duration, average mfcc distribution, average amplitude distribution)  (Yaggy)

### 1. bird image

### 2. feature visualization

### 3. statistics (Yaggy)

Load the all data.

In [ ]:
data = pd.read_csv("/home/yangya/桌面/project/train_metadata.csv/train_metadata.csv",engine='python')
data.head()

In [ ]:
all_class_labels=list(data.index)

### Visulaize the distribution of each species (pie chart)

In [ ]:
datagroup = data.groupby("primary_label").count()
datagroup

In [ ]:
print(type(datagroup))

In [ ]:
sum_ = datagroup['secondary_labels'].sum()
distri = datagroup['secondary_labels']/float(sum_)
class_labels = list(datagroup.index)

In [ ]:
y=np.array([35,25,25,15])
plt.pie(distri)#autopct="%.2f%%")#labels=class_labels)
plt.show()

### Visualization of Duration time(seconds) distribution of each species
In this part, we visualize the duration of each species. Here, we used a violin plot to visualize the distribution of duration for each species, because there are 264 categories in total and one plot cannot put down all of them, so we divided the plot into 7 subplots so that the data can be viewed a little more clearly. The violin plot is a good representation of the density, frequency, median, and quantile of each array. As you can see from the graph, the distribution of categories in this dataset is not quite balanced, in line with our visualization of category distribution in the previous section, where some categories have a larger number of violins and some categories have a smaller number of violins and a smaller area. However, from the figure, the distribution of duration of each category is relatively concentrated in the same location.

In [ ]:
def getduration(i): 
    """
    display waveform of a given speech sample
    :param sample_name: speech sample name
    :param fs: sample frequency
    :return:
    """
    file_prefix = "/home/yangya/桌面/project/train_audio/"
    idx = i
    sample = data.iloc[idx]
    path = file_prefix + sample["filename"]
    samples, sr = librosa.load(path, sr=16000)
    # samples = samples[6000:16000]
    avg_amp =librosa.get_duration(y=samples,sr=sr)
    return avg_amp

In [ ]:
getduration(3)

In [ ]:
data_avg1 = {}
for i in range(len(class_labels)):
    data_avg1[class_labels[i]] = []
for i in range(len(data)):
    temp = getduration(i)
    label = data['primary_label'][i]
    data_avg1[label].append(temp)

In [ ]:
df1_1=[]
for i in range(len(class_labels)):
    df1_1.append(data_avg1[class_labels[i]])
print(len(class_labels))
print(len(df1_1))
df1_2=df1_1[:40]
df1_3=df1_1[41:80]
df1_4=df1_1[81:120]
df1_5=df1_1[121:160]
df1_6=df1_1[161:220]
df1_7=df1_1[221:264]

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df1_2)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df1_3)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df1_4)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df1_5)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df1_6)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df1_7)

## Visualization of average mfcc distribution of each species
In this part, we visualize the average mfcc of each species. The figures tell that the distribution of average mfcc is signicantly different. So we use the mfccs feature to do the classification using traditional classifiers.

In [ ]:
def get_mean_mfcc(i):
    ile_prefix = "/home/yangya/桌面/project/train_audio/"
    idx = i
    sample = data.iloc[idx]
    path = file_prefix + sample["filename"]
    y, sr = librosa.load(path)

# 计算 MFCC 特征
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return (mfccs.mean())

In [ ]:
data_avg2 = {}
for i in range(len(class_labels)):
    data_avg2[class_labels[i]] = []
for i in range(len(data)):
    temp = get_mean_mfcc(i)
    label = data['primary_label'][i]
    data_avg2[label].append(temp)

In [ ]:
df2_1=[]
for i in range(len(class_labels)):
    df2_1.append(data_avg2[class_labels[i]])

In [ ]:
print(len(class_labels))
print(len(df2_1))
df2_2=df2_1[:40]
df2_3=df2_1[41:80]
df2_4=df2_1[81:120]
df2_5=df2_1[121:160]
df2_6=df2_1[161:220]
df2_7=df2_1[221:264]

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df2_2)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df2_3)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df2_4)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df2_5)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df2_6)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df2_7)

### Visualization of average Amplitude distribution of each species
In this part, we visualize the average amplitude distribution of each species. The figures tell that the average amplitude vary little. In the amplitude distribution chart, the median for all species is around 0.03,0.02.

In [ ]:
def dcalavgamplitude(i): 
    """
    display waveform of a given speech sample
    :param sample_name: speech sample name
    :param fs: sample frequency
    :return:
    """
    file_prefix = "/home/yangya/桌面/project/train_audio/"
    idx = i
    sample = data.iloc[idx]
    path = file_prefix + sample["filename"]
    samples, sr = librosa.load(path, sr=16000)
    # samples = samples[6000:16000]
    avg_amp =abs(samples).mean()
    return avg_amp
    # 
    # print(len(samples), sr)
    # time = np.arange(0, len(samples)) * (1.0 / sr)
    # plt.plot(time, samples)
    # plt.title("time v.s. amplitude")
    # plt.xlabel("time(s)")
    # plt.ylabel("Amplitude")
    # # plt.savefig("your dir\语音信号时域波形图", dpi=600)
    # plt.show()
    # 

In [ ]:
data_avg = {}
for i in range(len(class_labels)):
    data_avg[class_labels[i]] = []
for i in range(len(data)):
    temp = dcalavgamplitude(i)
    label = data['primary_label'][i]
    data_avg[label].append(temp)

In [ ]:
#data_1 = {'labels':all_class_labels,'amplitude':mean_avg}

#df_1 = pd.DataFrame(data_avg)
df_1=[]
for i in range(len(class_labels)):
    df_1.append(data_avg[class_labels[i]])

In [ ]:
print(len(class_labels))
print(len(df_1))
df_2=df_1[:40]
df_3=df_1[41:80]
df_4=df_1[81:120]
df_5=df_1[121:160]
df_6=df_1[161:220]
df_7=df_1[221:264]

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df_2)

In [ ]:

#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df_3)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df_4)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df_5)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df_6)

In [ ]:
#m = [2,3,4,5,6,7,8,9]
_, ax = plt.subplots(figsize=(100,10))
#df_1 = pd.DataFrame(c)
sns.violinplot(ax=ax, data=df_7)

# **Traditional Classifier**
## Outline
### 1. Features: MFCCs
- Data Preprocessing
- Classifiers
     - **KMeans Clustering**
      - Naive Bayes Multinomial BOW
      - Naive Bayes Multinomial TF-IDF
      - Suport Vector Machine(SVM) TF-IDF using linear kernel
      - Support Vector Machine (SVM) TF-IDF using linear kernel with cross-validation
      - Support Vector Machine (SVM) TF-IDF using RBF kernel
      - Support Vector Machine (SVM) TF-IDF using RBF kernel with cross-validation
      - Gradient Boosting TF-IDF with cross-validation
      - Random Forest TF-IDF with cross-validation
      - Logistic Regression TF-IDF
      - Logistic Regression TF-IDF with cross-validation

      - Use ***PCA*** to reduce the feature dimensionality
        - SVM (kernel='rbf')  (features dimensionality reduced by PCA)
     - **MeanShitft Clustering**
      - SVM(kernel='rbf')
      - Use **PCA** to reduce the feature dimensionality 
        - SVM (kernel='rbf')  (features dimensionality reduced by PCA)
     - **GMM Clustering**
      - SVM(kernel='rbf')
      - Use **PCA** to reduce the feature dimensionality 
        - SVM (kernel='rbf')  (features dimensionality reduced by PCA)
     - **Spectral Clustering**
      - SVM(kernel='rbf')
      - Use **PCA** to reduce the feature dimensionality 
        - SVM (kernel='rbf')  (features dimensionality reduced by PCA)

Firstly, we use panda.read_csv to read the whole data and it returns data with type of Dataframe. And then ,we define a function to convert the audio file into mfccs features by using functions implemented by librosa. After this, we use clustering methods to find the most distinct mfccs features and map the mfccs features of each audio to the index of clustering centers. We use the processed data to do the classification.

Here are the analysis of our traditional machine learning classifier.
In the beginning, we split the whole data into traindata and test data, and use the traindata to train the classifer with cross-validation, then we use the testdata to evaluate the classifier. We create the bag of audios and convert the auido file into the index of audio file in the bag. We get two representations of the audio file, thanks to feature_extraction.text.TfidfTransformer, which are BOW and TF-IDF. BoW just consider the frequency of a 'word' without the importance of a 'word',while the TF-IDF considers both the frequency and importance. We firstly use Naive Bayes Multinomial classfier to classify the species with BOW and TF-IDF respectively, and the features TF-IDF works better. So in the next experiments, we use TF-IDF to do the classification. Though we know when TF-IDF works better with Naive Bayes Multinomial, it does not mean it will works better with other classifiers, we just simplify the problem. So that wo needn't to run so many classifer which just cost my time with no other techniques. 
In the next experiments, we use SVM(kernel = linear), SVM(kernel=rbf), Gradient Boosting , Random Forest and LR with cross-validation to do the classification. Among all the classifer, SVM(kernel=rbf) works best, so we use the classifer to do the classification after we use PCA to reduce the dimensionality of TF-IDF features. We use PCA to reduce the dimensionality after each clustering method. In order to find the if the performance affected by clustering method, we try different clustering method, which includs KMeans, Meanshift, GMM,and Spectral Clustering method. We find that the clustering method has little influence on the performance.BTW，SVM(kernel=rbf）works best when using KMeans and TF-IDF , so when using different clustering method, we use SVM(kernel = rbf) to evaluate the peroformance. However, the performance is ver bad. In order to improve the performance, we consider using DNN models. We will analysis it in the next part.

## Data Preprocessing

In [ ]:
data = pd.read_csv("/home/yangya/桌面/project/train_metadata.csv/train_metadata.csv",engine='python')
data.head()

Split the data into traindata and test data.

In [ ]:
train_df, test_df = train_test_split(data, test_size=0.2, random_state=101)

A test to convert the ogg file into mfccs features.

In [ ]:

# 加载一个音频文件
file_prefix = "/home/yangya/桌面/project/train_audio/"
idx = 5
sample = data.iloc[idx]
path = file_prefix + sample["filename"]
y, sr = librosa.load(path)
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

In [ ]:
print(mfccs)
print(np.array(mfccs).T.shape)#将计算得到的mfccs保存成转置形式

Data preprocessing.

In [ ]:
#convert ogg file into mfccs
def convert_to_mfccs(data):
    mfccs_res = []
    file_prefix = "/home/yangya/桌面/project/train_audio/"
    for i in range(len(data)):
        idx =i 
        sample = data.iloc[idx]
        path = file_prefix + sample["filename"]
        y, sr = librosa.load(path)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfccs_res.append(mfccs.T)
    return mfccs_res

In [ ]:
#get the mfccs feature 
train_mfccs = convert_to_mfccs(train_df)
test_mfccs= convert_to_mfccs(test_df)

In [ ]:
# compute delta MFCCs
def compute_delta_mfccs(mfccs):
    dmfccs = []
    for m in mfccs:
        tmp = m[1:] - m[0:-1]
        dm = np.hstack((m[0:-1], tmp))
        dmfccs.append(dm)
    return dmfccs

In [ ]:
#get the compute delta mfccs
train_dmfccs = compute_delta_mfccs(train_mfccs)
test_dmfccs = compute_delta_mfccs(test_mfccs)

In [ ]:
all_dmfccs = np.vstack(train_dmfccs)
print(all_dmfccs.shape)

In [ ]:
tagnames = data["primary_label"].unique()

In [ ]:
# convert list of tags into binary class labels
def tags2class(tags, tagnames):
    b = np.zeros(shape=(len(tags), len(tagnames)))
    for i,t in enumerate(tags):
        for j,n in enumerate(tagnames):
            if n in t:
                b[i,j] = 1
    return b

In [ ]:
# train_classes[i,j] = absence/presence of the j-th tag in the i-th sound
train_classes_ = tags2class(train_df['primary_label'], tagnames)
test_classes_ = tags2class(test_df['primary_label'], tagnames)

In [ ]:
train_classes = []
for i in range(len(train_classes_)):
    train_classes.append(np.argmax(train_classes_[i]))

In [ ]:
test_classes = []
for j in range(len(test_classes_)):
    test_classes.append(np.argmax(test_classes_[j]))

In [ ]:
print(np.array(train_classes).shape)
print(np.array(test_classes).shape)
#print(test_classes_)

## KMeans Clustering 

In [ ]:
# run k-means to build codebook
km = cluster.KMeans(n_clusters=100, random_state=4487)
km.fit(all_dmfccs[0::100])  # subsample by 10 to make it faster
km.cluster_centers_

In [ ]:
def bow_transform(model, mfccs):
    numwords = model.cluster_centers_.shape[0]
    bows = np.zeros((len(mfccs), numwords))
    for i in range(len(mfccs)):
        w = model.predict(mfccs[i])
        bw = np.bincount(w, minlength=numwords)
        bows[i,:] = bw
    return bows

BoW representation.

In [ ]:
train_bow = bow_transform(km, train_dmfccs)
test_bow = bow_transform(km,test_dmfccs)

In [ ]:
print(train_bow.shape)

TF-IDF representation.

In [ ]:
# convert to TF
tf_trans = feature_extraction.text.TfidfTransformer(use_idf=True, norm='l1')
train_Xtf = tf_trans.fit_transform(train_bow)
test_Xtf  = tf_trans.transform(test_bow)

Classifiers.

## Naive Bayes Multinomial BOW

In [ ]:
# Train Multinomial NB model
def trainMModel(a):
    mmodel = naive_bayes.MultinomialNB(alpha = a)
    mmodel.fit(train_bow,train_classes)
    
    testY = test_classes
    predY = mmodel.predict(test_bow)
    acc = metrics.accuracy_score(testY, predY)
    
    return acc

# Grid Search to find the best performance parameter setting (alpha)
def GridSearchMModel(start_a, num_a):
    best_a = start_a
    best_acc = 0.0
    for i in range(0, num_a):
        a = start_a + (i / (num_a - 1))
        tmp_acc = trainMModel(a)
        if tmp_acc > best_acc:
            best_a = a
            best_acc = tmp_acc
    return best_a, best_acc

best_a, best_acc = GridSearchMModel(0.0, 10001)
print("Parameter setting with best performance: alpha = {}, accuracy = {}".format(best_a, best_acc))

## Naive Bayes Multinomial TF-IDF 

In [ ]:
# Train Multinomial NB model
def trainMModel(a):
    mmodel = naive_bayes.MultinomialNB(alpha = a)
    mmodel.fit(train_Xtf,train_classes)
    
    testY = test_classes
    predY = mmodel.predict(test_Xtf)
    acc = metrics.accuracy_score(testY, predY)
    
    return acc

# Grid Search to find the best performance parameter setting (alpha)
def GridSearchMModel(start_a, num_a):
    best_a = start_a
    best_acc = 0.0
    for i in range(0, num_a):
        a = start_a + (i / (num_a - 1))
        tmp_acc = trainMModel(a)
        if tmp_acc > best_acc:
            best_a = a
            best_acc = tmp_acc
    return best_a, best_acc

best_a, best_acc = GridSearchMModel(0.0, 10001)
print("Parameter setting with best performance: alpha = {}, accuracy = {}".format(best_a, best_acc))

TF-IDF is better than BOW, so in the next experiments, we use TF-IDF as features to classify species

## Suport Vector Machine(SVM) using linear kernel

In [ ]:
# Using SVM with linear kernel
def trainSVM(c):
    clf = pipeline.Pipeline([('vect', feature_extraction.text.CountVectorizer()), ('tfidf', feature_extraction.text.TfidfTransformer()), ('clf', svm.SVC(C = c, kernel = 'linear'))])
    svm_clf = clf.fit(train_Xtf, train_classes)
    svm_predY = svm_clf.predict(test_Xtf)
    acc_svm = metrics.accuracy_score(test_classes, svm_predY)
    
    return acc_svm

# Grid Search to find the C with best performance
def GridSearchSVM(Cs):
    best_c = Cs[0]
    best_acc = 0.0
    for i in range(len(Cs)):
        tmp_acc = trainSVM(Cs[i])
        if tmp_acc > best_acc:
            best_c = Cs[i]
            best_acc = tmp_acc
    return best_c, best_acc

Cs = np.logspace(-5, 5, 50)
best_c, best_acc_svm = GridSearchSVM(Cs)
print("Parameter setting with best performance: C = {}, accuracy = {}".format(best_c, best_acc_svm))

## Support Vector Machine (SVM) using linear kernel with cross-validation

In [ ]:
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv = model_selection.GridSearchCV(svm.SVC(kernel = 'linear'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv.fit(train_Xtf , train_classes);

print("best params:", svmcv.best_params_)
# predict from the model
predY1 = svmcv.best_estimator_.predict(test_Xtf)

# calculate accuracy
acc1 = metrics.accuracy_score(test_classes, predY1)
print("test accuracy =", acc1)

## Support Vector Machine (SVM) using RBF kernel

In [ ]:
# Using SVM with linear kernel
def trainSVMRBF(c):
    clf = pipeline.Pipeline([('vect', feature_extraction.text.CountVectorizer()), ('tfidf', feature_extraction.text.TfidfTransformer()), ('clf', svm.SVC(C = c, kernel = 'rbf'))])
    svm_clf = clf.fit(train_Xtf, train_classes)
    svm_predY = svm_clf.predict(test_Xtf)
    acc_svm = metrics.accuracy_score(test_classes, svm_predY)
    
    return acc_svm

# Grid Search to find the C with best performance
def GridSearchSVMRBF(Cs):
    best_c = Cs[0]
    best_acc = 0.0
    for i in range(len(Cs)):
        tmp_acc = trainSVMRBF(Cs[i])
        if tmp_acc > best_acc:
            best_c = Cs[i]
            best_acc = tmp_acc
    return best_c, best_acc

Cs = np.logspace(-5, 5, 50)
best_c3, best_acc_svm3 = GridSearchSVMRBF(Cs)
print("Parameter setting with best performance: C = {}, accuracy = {}".format(best_c3, best_acc_svm3))

## Support Vector Machine (SVM) using RBF kernel with cross-validation

In [ ]:
# setup the list of parameters to try
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv2 = model_selection.GridSearchCV(svm.SVC(kernel = 'rbf'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv2.fit(train_Xtf, train_classes)

print("best params:", svmcv2.best_params_)

# predict from the model
predY2 = svmcv2.best_estimator_.predict(test_Xtf)

# calculate accuracy
acc2 = metrics.accuracy_score(test_classes, predY2)
print("test accuracy =", acc2)

## Gradient Boosting

In [ ]:
import xgboost as xgb
# Gradient Boosting

paramsampler= {    
    "colsample_bytree": stats.uniform(0.7, 0.3),  
    "gamma":            stats.uniform(0, 0.5),    
    "max_depth":        stats.randint(2, 6),      
    "subsample":        stats.uniform(0.6, 0.4),  
    "learning_rate":    stats.uniform(.001,1),    
    "n_estimators":     stats.randint(10, 1000),
}

#X_train, X_test, y_train, y_test = model_selection.train_test_split(trainXtf, trainY, test_size = 0.2, random_state = 0)
xclf = xgb.XGBClassifier(objective = "multi:softmax", random_state = 4487)
xgbcv = model_selection.RandomizedSearchCV(xclf, param_distributions = paramsampler, random_state = 4487, n_iter = 200, cv = 5, verbose = 1, n_jobs = -1)
xgbcv.fit(train_Xtf , train_classes)

print("best params:", xgbcv.best_params_)

In [ ]:
xgb_predY = xgbcv.best_estimator_.predict(test_Xtf)
acc_xgb = metrics.accuracy_score(test_classes, xgb_predY)
print("Gradient Boosting classifier accuracy: {}".format(acc_xgb))

## Random Forest

In [ ]:
paramsampler = {#'max_features': stats.uniform(0,1.0),
                 'max_depth':         stats.randint(1,5),
                 'min_samples_split': stats.uniform(0,0.5), 
                 'min_samples_leaf':  stats.uniform(0,0.5),
               }

rfrcv = model_selection.RandomizedSearchCV(
                            ensemble.RandomForestClassifier(n_estimators = 100, random_state = 4487, n_jobs = -1),
                            param_distributions = paramsampler, 
                            random_state = 4487, n_iter = 1000, cv = 5, 
                            verbose = 1, n_jobs = -1)

rfrcv.fit(train_Xtf, train_classes);

print("best params:", rfrcv.best_params_)

In [ ]:
rf_predY = rfrcv.best_estimator_.predict(test_Xtf)
acc_rf = metrics.accuracy_score(test_classes, rf_predY)
print("Random Forest classifier accuracy: {}".format(acc_rf))

## Logistic Regression

In [ ]:
# Using LR
def trainLR(c):
    logreg = pipeline.Pipeline([('vect', feature_extraction.text.CountVectorizer()), ('tfidf', feature_extraction.text.TfidfTransformer()), ('clf', linear_model.LogisticRegression(n_jobs = -1, C = c, solver = 'saga'))])
    lr_clf = logreg.fit(train_Xtf, train_classes)
    lr_predY = lr_clf.predict(test_Xtf)
    acc_lr = metrics.accuracy_score(test_classes, lr_predY)
    
    return acc_lr

# Grid Search to find the C with best performance
def GridSearchLR(Cs):
    best_c = Cs[0]
    best_acc = 0.0
    for i in range(len(Cs)):
        tmp_acc = trainLR(Cs[i])
        if tmp_acc > best_acc:
            best_c = Cs[i]
            best_acc = tmp_acc
    return best_c, best_acc

Cs = logspace(-5, 5, 50)
best_c_lr, best_acc_lr = GridSearchLR(Cs)
print("Parameter setting with best performance: C = {}, accuracy = {}".format(best_c_lr, best_acc_lr))

## Logistic Regression with cross-validation

In [ ]:
Cs = logspace(-5, 5, 50)

# setup the cross-validation object
lrcv_clf = pipeline.Pipeline([('vect', feature_extraction.text.CountVectorizer()), ('tfidf', feature_extraction.text.TfidfTransformer()), ('clf', linear_model.LogisticRegressionCV(n_jobs = -1, Cs = Cs, solver = 'saga', max_iter = 10000))])
lrcv = lrcv_clf.fit(train_Xtf, train_classes)

lr_predY = lrcv.predict(test_Xtf)
acc_lrcv = metrics.accuracy_score(test_classes, lr_predY)

print("Accuracy of LR model with cross-validation: ", acc_lrcv)

## Use PCA to reduce dimensionality

In [ ]:
pca_model = decomposition.TruncatedSVD(n_components=90)
pca_model.fit(train_Xtf)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# This function plot the PCA curve
def plot_exp_ratio(ratio, title):
    explain_fig = plt.figure()
    idx = np.where(ratio > 0.95)[0]#[0]
    print("95% ratio when components are {}".format(idx))
    #plt.title(title)
    #plt.plot(ratio)

In [ ]:
#print((np.cumsum(pca_model.explained_variance_ratio_)).shape)
plot_exp_ratio(np.cumsum(pca_model.explained_variance_ratio_), 
               "Explained Variance Ratio(PCA)")

## Covert the tran_Xtf and test_xtf to reduced dimensionality

In [ ]:
pca_500To285 = decomposition.TruncatedSVD(n_components=76)
pca_500To285.fit(train_Xtf)

train_Xtfpca = pca_500To285.transform(train_Xtf)
test_Xtfpca = pca_500To285.transform(test_Xtf)

In [ ]:
print(train_Xtfpca.shape)
print(test_Xtfpca.shape)

## Use SVM (kernel='rbf') to estimate [features dimensionality reduced by PCA]

In [ ]:
# setup the list of parameters to try
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv2 = model_selection.GridSearchCV(svm.SVC(kernel = 'rbf'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv2.fit(train_Xtfpca, train_classes)

print("best params:", svmcv2.best_params_)

# predict from the model
predY2 = svmcv2.best_estimator_.predict(test_Xtfpca)

# calculate accuracy
acc2 = metrics.accuracy_score(test_classes, predY2)
print("test accuracy =", acc2)

# Mean-Shift Clustering

In [ ]:
# run k-means to build codebook
km = cluster.MeanShift(bandwidth=5, bin_seeding=True, n_jobs=-1)#cluster.KMeans(n_clusters=100, random_state=4487)
km.fit(all_dmfccs[0::100])  # subsample by 10 to make it faster
km.cluster_centers_

In [ ]:
def bow_transform(model, mfccs):
    numwords = model.cluster_centers_.shape[0]
    bows = np.zeros((len(mfccs), numwords))
    for i in range(len(mfccs)):
        w = model.predict(mfccs[i])
        bw = np.bincount(w, minlength=numwords)
        bows[i,:] = bw
    return bows

trainmeanshift_bow = bow_transform(km, train_dmfccs)
testmeanshift_bow = bow_transform(km,test_dmfccs)

In [ ]:
# convert to TF
tf_trans = feature_extraction.text.TfidfTransformer(use_idf=True, norm='l1')
trainmeanshift_Xtf = tf_trans.fit_transform(trainmeanshift_bow )
testmeanshift_Xtf  = tf_trans.transform(testmeanshift_bow)

## SVM(kernel='rbf') [features extracted by Meanshift method]

In [ ]:
# setup the list of parameters to try
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv2 = model_selection.GridSearchCV(svm.SVC(kernel = 'rbf'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv2.fit(trainmeanshift_Xtf , train_classes)

print("best params:", svmcv2.best_params_)

# predict from the model
predY2 = svmcv2.best_estimator_.predict(testmeanshift_Xtf)

# calculate accuracy
acc2 = metrics.accuracy_score(test_classes, predY2)
print("test accuracy =", acc2)

## Use PCA to reduce dimensionality and then use SVM（rbf kernel） to do the classification

In [ ]:
pca_500To285 = decomposition.TruncatedSVD(n_components=50)
pca_500To285.fit(train_Xtf)

train_Xtfpca = pca_500To285.transform(train_Xtf)
test_Xtfpca = pca_500To285.transform(test_Xtf)

In [ ]:
# setup the list of parameters to try
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv2 = model_selection.GridSearchCV(svm.SVC(kernel = 'rbf'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv2.fit(trainmeanshift_Xtf , train_classes)

print("best params:", svmcv2.best_params_)

# predict from the model
predY2 = svmcv2.best_estimator_.predict(testmeanshift_Xtf)

# calculate accuracy
acc2 = metrics.accuracy_score(test_classes, predY2)
print("test accuracy =", acc2)

# GMM Clustering

In [ ]:
# run k-means to build codebook
km = mixture.GaussianMixture(n_components=100, covariance_type='full', random_state=4487, n_init=10)
km.fit(all_dmfccs[0::100])  # subsample by 10 to make it faster
km.cluster_centers_

In [ ]:
def bow_transform(model, mfccs):
    numwords = model.cluster_centers_.shape[0]
    bows = np.zeros((len(mfccs), numwords))
    for i in range(len(mfccs)):
        w = model.predict(mfccs[i])
        bw = np.bincount(w, minlength=numwords)
        bows[i,:] = bw
    return bows

trainGMM_bow = bow_transform(km, train_dmfccs)
testGMM_bow = bow_transform(km,test_dmfccs)

In [ ]:
# convert to TF
tf_trans = feature_extraction.text.TfidfTransformer(use_idf=True, norm='l1')
trainGMM_Xtf = tf_trans.fit_transform(trainGMM_bow )
testGMM_Xtf  = tf_trans.transform(testGMM_bow)

## SVM(kernel='rbf') [features extracted by GMM method]

In [ ]:
# setup the list of parameters to try
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv2 = model_selection.GridSearchCV(svm.SVC(kernel = 'rbf'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv2.fit(trainGMM_Xtf , train_classes)

print("best params:", svmcv2.best_params_)

# predict from the model
predY2 = svmcv2.best_estimator_.predict(testGMM_Xtf)

# calculate accuracy
acc2 = metrics.accuracy_score(test_classes, predY2)
print("test accuracy =", acc2)

## Use PCA to reduce dimensionality from features extracted by GMM clustering and then use SVM（rbf kernel） to do the classification

In [ ]:
pca_500To285 = decomposition.TruncatedSVD(n_components=50)
pca_500To285.fit(trainGMM_Xtf)

train_Xtfpca = pca_500To285.transform(trainGMM_Xtf)
test_Xtfpca = pca_500To285.transform(testGMM_Xtf)

In [ ]:
# setup the list of parameters to try
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv2 = model_selection.GridSearchCV(svm.SVC(kernel = 'rbf'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv2.fit(train_Xtfpca , train_classes)

print("best params:", svmcv2.best_params_)

# predict from the model
predY2 = svmcv2.best_estimator_.predict(test_Xtfpca)

# calculate accuracy
acc2 = metrics.accuracy_score(test_classes, predY2)
print("test accuracy =", acc2)

# Spectral Clustering

In [ ]:
# run k-means to build codebook
km = luster.SpectralClustering(n_clusters=4100, affinity='rbf', gamma=1.0, assign_labels='discretize', n_jobs=-1)
km.fit(all_dmfccs[0::100])  # subsample by 10 to make it faster
km.cluster_centers_

In [ ]:
def bow_transform(model, mfccs):
    numwords = model.cluster_centers_.shape[0]
    bows = np.zeros((len(mfccs), numwords))
    for i in range(len(mfccs)):
        w = model.predict(mfccs[i])
        bw = np.bincount(w, minlength=numwords)
        bows[i,:] = bw
    return bows

trainS_bow = bow_transform(km, train_dmfccs)
testS_bow = bow_transform(km,test_dmfccs)

In [ ]:
pca_500To285 = decomposition.TruncatedSVD(n_components=50)
pca_500To285.fit(trainGMM_Xtf)

train_Xtfpca = pca_500To285.transform(trainGMM_Xtf)
test_Xtfpca = pca_500To285.transform(testGMM_Xtf)

## Use PCA to reduce dimensionality of features extracted by Spectral clustering method and then use SVM（rbf） to do the classification

In [ ]:
# setup the list of parameters to try
paramgrid = {'C': np.logspace(-5, 5, 50)}

print(paramgrid)

# setup the cross-validation object
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds
svmcv2 = model_selection.GridSearchCV(svm.SVC(kernel = 'rbf'), paramgrid, cv=5, n_jobs=-1, verbose=True)

# run cross-validation (train for each split)
svmcv2.fit(trainS_Xtf , train_classes)

print("best params:", svmcv2.best_params_)

# predict from the model
predY2 = svmcv2.best_estimator_.predict(testS_Xtf)

# calculate accuracy
acc2 = metrics.accuracy_score(test_classes, predY2)
print("test accuracy =", acc2)